In [ ]:
import pandas as pd

statements = {
    1: 'b=1',
    2: 'c=2',
    3: 'a=b+c',
    4: 'd=a-b',
    5: 'd=c+d',
    6: 'c=b+c',
    7: 'e=a-b',
    8: 'd=b+c',
    9: 'e=e+1',
    10: 'b=c*d',
    11: 'c=b-d'
}

blocks = {
    1: [1, 2],
    2: [3, 4],
    3: [5],
    4: [6, 7],
    5: [8, 9],
    6: [10, 11]
}

graph = {
    1: [],
    2: [1, 4],
    3: [2, 5],
    4: [2, 3],
    5: [3],
    6: [4]
}

var = {}
for i in range(1, len(blocks) + 1):
    s = set()
    for x in blocks[i]:
        s.add(statements[x][0])
    var[i] = s

kill = {}
for x in var:
    s = set()
    for y in statements:
        if y not in blocks[x]:
            if statements[y][0] in var[x]:
                s.add(y)
    kill[x] = s

gen = {}
predecessor = {}
for x in blocks:
    gen[x] = set(blocks[x])
for x in graph:
    predecessor[x] = set(graph[x])

IN = {}
OUT = {}
for x in blocks:
    IN[x] = set()
    OUT[x] = gen[x]

def display(IN, OUT):
    lin = [IN[i] for i in IN]
    lout = [OUT[i] for i in OUT]
    block = [i for i in blocks]
    data = {'block': block, 'IN': lin, 'OUT': lout}
    df = pd.DataFrame(data)
    print(df)

lin = [IN[i] for i in IN]
lout = [OUT[i] for i in OUT]
block = [i for i in blocks]
gens = [gen[i] for i in gen]
kills = [kill[i] for i in kill]
pl = [predecessor[i] for i in predecessor]
data = {'block': block, 'gen': gens, 'kill': kills, 'predecessor': pl, 'IN': lin, 'OUT': lout}
df = pd.DataFrame(data)
print(df)

change = True
it = 0
while change:
    change = False
    it += 1
    nOUT = {}
    for x in blocks:
        IN[x] = set()
        for i in predecessor[x]:
            IN[x] = IN[x].union(OUT[i])

        oldout = OUT[x]
        nOUT[x] = gen[x].union(IN[x] - kill[x])
        if nOUT[x] != oldout:
            change = True
    OUT = nOUT
    print("\n")
    print("iteration", it)
    display(IN, OUT)


   block       gen         kill predecessor  IN       OUT
0      1    {1, 2}  {10, 11, 6}          {}  {}    {1, 2}
1      2    {3, 4}       {8, 5}      {1, 4}  {}    {3, 4}
2      3       {5}       {8, 4}      {2, 5}  {}       {5}
3      4    {6, 7}   {9, 2, 11}      {2, 3}  {}    {6, 7}
4      5    {8, 9}    {4, 5, 7}         {3}  {}    {8, 9}
5      6  {10, 11}    {1, 2, 6}         {4}  {}  {10, 11}


iteration 1
   block            IN                 OUT
0      1            {}              {1, 2}
1      2  {1, 2, 6, 7}  {1, 2, 3, 4, 6, 7}
2      3  {8, 9, 3, 4}           {9, 3, 5}
3      4     {3, 4, 5}     {3, 4, 5, 6, 7}
4      5           {5}              {8, 9}
5      6        {6, 7}         {10, 11, 7}


iteration 2
   block                        IN                    OUT
0      1                        {}                 {1, 2}
1      2     {1, 2, 3, 4, 5, 6, 7}     {1, 2, 3, 4, 6, 7}
2      3  {1, 2, 3, 4, 6, 7, 8, 9}  {1, 2, 3, 5, 6, 7, 9}
3      4  {1, 2, 3, 4, 5, 6, 7, 9